# *Notebook* à utiliser pour faire le travail pratique # 3 sur l'analyse d'incidents.





## Importation des bibliothèques

In [1]:
# Importation des bibliothèques 
import torch
import json
from transformers import pipeline
from __future__ import print_function
from collections import Counter
import string
import re
import json



In [2]:
# Vérifiez si un GPU est disponible et utilisez-le, sinon utilisez le CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Chargements des données et questions 

In [3]:
with open('data/dev_examples.json', 'r') as file:
  data = json.load(file)

liste_questions =[
  [
    "what happened ?",
    "describe the event that occured ?"
  ],
  [
    
    "What was the employee doing right before the incident, and what led to it?",
    "what was the main activity?",
    "what was the activity?"
    

  ],
  [
    "Who is the main person involved in this incident?",
    "Can you identify the individual involved in this incident?"
  ],
  [
     "Where did this incident take place?",
     "Can you specify the exact location where the incident occurred?"
  ],
  [
    "When did this incident occur?",
    "Can you provide the precise date and time of the incident?"

  ],
  [
    "What is the  cause ?",
    "what is the cause of incident?"
    "describe the cause of incident"
    "Explain what caused this incident."
  ],
  [
     "What equipment was in this incident?",
     "Can you name the tools or equipment used at the time of the incident?"
  ],
  [
    
    "what kind of injury was there?",
    "Describe the physical damages  from the incident."
    
  ],
  [
    "Who was injured in this incident?",
    "Can you identify the person who suffered injuries?"
  ],
  [
    "Which body parts were affected by the injuries?",
    "Describe the part of the body affected by the incident."
  ],
  [
    
    "who was dead?",
    "wich person died?"
    
  ],
  [
    "Were there any substances involved in this incident?",
    "Name the substances that played a role in this incident."
  ]
  
]

## Configuration du modèle de question-réponse

In [4]:
model_name = "deepset/roberta-base-squad2"

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
#Fonction pour exécuter le modèle de question-réponse
def question_model(question,context) : 
    QA_input = {
        'question': question,
        'context': context
    }
    res = nlp(QA_input)
    return res['answer']

## les fonctions d'évaluation

In [5]:



def normalize_answer(s):
    """Mettre en minuscule et retirer la ponctuation, des déterminants and les espaces."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    """Normalise les 2 textes, trouve ce qu'il y a en comment et estime précision, rappel et F1."""
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if len(ground_truth_tokens) == 0 or len(prediction_tokens) == 0:
        return int(ground_truth_tokens == prediction_tokens)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth): 
    """Vérifie si les 2 textes sont quasi-identiques."""
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    """La fonction princiaple. Important de noter que ground_truths est une liste 
       parce qu'il peut y avoir plusieurs réponses possibles."""
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)

## Analyse d'incidents et des questions

In [ ]:
#liste des clés d'arguments pour les incidents
arguments_keys = ['EVENT', 'ACTIVITY', 'WHO', 'WHERE', 'WHEN', 'CAUSE', 'EQUIPMENT', 'INJURY', 'INJURED', 'BODY-PARTS', 'DEATH', 'SUBSTANCE']
#boucle sur les données et les questions
for d in data[:10]:
    context = d['text']
    arguments = d['arguments']
    print("context : ", context)
    for key, questions in zip(arguments_keys, liste_questions):
        if key in arguments:
            print("Arguments : ", arguments[key])
            for i, question in enumerate(questions):
                res = question_model(question, context)

                exact_match = metric_max_over_ground_truths(exact_match_score, res, arguments[key])
                f1_value = metric_max_over_ground_truths(f1_score, res, arguments[key])
                
                print(f"Q{i+1} : {question}")
                print(f"R{i+1} : {res}")
                print('Exact match:', exact_match, '\nF1:', f1_value)
                print()






        




## Choix du modèle et pertinence de l'approche

Nous avons opté pour le modèle Roberta, plus précisément deepset/roberta-base-squad2, en raison de plusieurs avantages significatifs qui le rendent particulièrement adapté à la tâche de Question-Réponse (QA) dans le cadre de l'analyse d'incidents.

Architecture pré-entraînée performante : Les modèles Roberta sont basés sur une architecture de transformateur pré-entraînée, qui a démontré une forte capacité à capturer des représentations sémantiques complexes. 
Cela se traduit par une meilleure compréhension du contexte et une réponse plus précise aux questions posées.

En utilisant un modèle pré-entraîné comme deepset/roberta-base-squad2, nous pouvons profiter d'une base solide tout en ayant la flexibilité d'ajuster le modèle pour des domaines spécifiques, comme notre analyse d'incidents. Cela permet une personnalisation efficace pour des contextes particuliers.

le choix de Roberta repose sur son succès antérieur dans le domaine du QA, sa robustesse face à des contextes variés, et son potentiel d'adaptation aux exigences spécifiques de l'analyse d'incidents.

## Évaluation de l'Impact de Différentes Formulations de Questions 
Formulations Réussies:
les questions ouvertes telles que "what happened?" ont généré des réponses précises, offrant une vue d'ensemble de l'événement.
La question "Where did this incident take place?" a conduit à une réponse précise, identifiant correctement le lieu comme "railroad bridge overpass."

Formulations Problématiques:

Les questions complexes comme "What is the cause? Describe the cause of the incident. Explain what caused this incident." ont parfois conduit à des réponses moins claires.

Influence de la Complexité de la Question:

Les formulations simples du type "Who?" ou "When?" ont bien fonctionné, tandis que des questions complexes ont parfois abouti à des réponses moins cohérentes.

Impact sur la Cohérence des Réponses:

Des variations dans les formulations ont influencé la cohérence. Par exemple, "What happened?" et "Describe the event that occurred?" ont obtenu des réponses similaires.